In [6]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
import random
import re


# for reference:
acronym_meanings = {
    'CC': 'Coordinating conjunction',
    'CD': 'Cardinal number',
    'DT': 'Determiner',
    'EX': 'Existential there',
    'FW': 'Foreign word',
    'IN': 'Preposition or subordinating conjunction',
    'JJ*': 'Adjective',
    'MD': 'Modal',
    'NN*': 'Noun',
    'PDT': 'Predeterminer',
    'POS': 'Possessive ending',
    'PRP*': 'Pronoun',
    'RB*': 'Adverb',
    'RP': 'Particle',
    'SYM': 'Symbol',
    'TO': 'To',
    'UH': 'Interjection',
    'VB*': 'Verb',
    'WDT': 'Wh-determiner',
    'WP*': 'Wh-pronoun',
    'WRB': 'Wh-adverb'
}

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NYCTA-P6\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\NYCTA-P6\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NYCTA-P6\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
text = "Cultivated peaches are divided into clingstones and freestones, depending on whether the flesh sticks to the stone or not; both can have either white or yellow flesh. Peaches with white flesh typically are very sweet with little acidity, while yellow-fleshed peaches typically have an acidic tang coupled with sweetness, though this also varies greatly. Both colors often have some red on their skins. Low-acid, white-fleshed peaches are the most popular kinds in China, Japan, and neighbouring Asian countries, while Europeans and North Americans have historically favoured the acidic, yellow-fleshed cultivars. Peach trees are relatively short-lived as compared with some other fruit trees. In some regions orchards are replanted after 8 to 10 years, while in others trees may produce satisfactorily for 20 to 25 years or more, depending upon their resistance to diseases, pests, and winter damage."
text2 = "A wave of Confederate surrenders followed. On April 14, just five days after Lee's surrender, Lincoln was assassinated. As a practical matter, the war ended with the May 26 surrender of the Department of the Trans-Mississippi but the conclusion of the American Civil War lacks a clear and precise historical end date. Confederate ground forces continued surrendering past the May 26 surrender date until June 23. By the end of the war, much of the South's infrastructure was destroyed, especially its railroads. The Confederacy collapsed, slavery was abolished, and four million enslaved black people were freed. The war-torn nation then entered the Reconstruction era in an attempt to rebuild the country, bring the former Confederate states back into the United States, and grant civil rights to freed slaves."
text3 = "In October 2006, YouTube was bought by Google for $1.65 billion.[11] Google's ownership of YouTube expanded the site's business model, expanding from generating revenue from advertisements alone to offering paid content such as movies and exclusive content produced by YouTube. It also offers YouTube Premium, a paid subscription option for watching content without ads. YouTube also approved creators to participate in Google's AdSense program, which seeks to generate more revenue for both parties. YouTube reported revenue of $29.2 billion in 2022.[12] In 2021, YouTube's annual advertising revenue increased to $28.8 billion, an increase in revenue of 9 billion from the previous year.[1]"
# text = text3



# the parts of speech that are allowed in the output
allowed_pos = ["JJ", "RB", "VB", "NN"]


result_tuples = []
chosen = []

# pops is used to limit the number of loops. Sometimes it bugs out if max_pops is set too high.
max_pops = 25
global pops
pops = 0





def text2tokens(text):
    bad_tokens = word_tokenize(text)

# not sure if stopwords are good or bad--a lot are already avoided due to "allowed_pos"
#     stop_words = set(stopwords.words('english'))
#     bad_tokens = []
#     for word in bad_tokens:
#         if word.lower() not in stop_words:
#             bad_tokens.append(word)

    # regular expression that matches only word characters. Prevents stuff like "]" which is a noun for some reason.
    pattern = re.compile(r'\w+')
    
    global tokens
    tokens = []
    for token in bad_tokens:
        if pattern.match(token):
            tokens.append(token)


def get_word():
    global choice, new_word, new_word_index, new_word_pos
    choice = random.choice(range(len(tokens)))
    if choice in chosen: # prevents the same word from being used twice. (maybe obsolete now)
        get_word()
    chosen.append(choice)
    new_word = tokens[choice]
    new_word_index = choice
    new_word_pos = pos_tag([new_word])[0][1][:2] # syntax: pos_tag generates a list of a tuple. This grabs the tuple, then grabs the 2nd element which is the POS as a string, then grabs the first 2 letters.
    
    
    
# this function is generalizable to any POS sequences i believe
def sequence_check():
    """Checks to see if there is an sequence like 'adjective then noun' around the chosen word. 
    If so, then it adds both words to the results list."""
    
    global new_word, new_word_index, new_word_pos
    if choice > 0 and choice < len(tokens) - 1: # avoids list index out of range errors
        prior_word = tokens[choice-1]
        prior_word_index = choice-1
        prior_word_pos = pos_tag([prior_word])[0][1][:2]

        next_word = tokens[choice+1]
        next_word_index = choice+1
        next_word_pos = pos_tag([next_word])[0][1][:2]
    
    # adj-noun-verb
        if prior_word_pos == "JJ" and new_word_pos == "NN" and next_word_pos =="VB":
            new_word = prior_word
            new_word_index = prior_word_index
            new_word_pos = prior_word_pos
            check_and_insert()
            new_word = next_word
            new_word_index = next_word_index
            new_word_pos = next_word_pos
            check_and_insert()
            print("adj-noun-verb worked :)")
    
    # adj-noun
        elif prior_word_pos == "JJ" and new_word_pos == "NN":
            new_word = prior_word
            new_word_index = prior_word_index
            new_word_pos = prior_word_pos
            check_and_insert()
            print("adj-noun worked :)")

        elif new_word_pos == "JJ" and next_word_pos == "NN":
            new_word = next_word
            new_word_index = next_word_index
            new_word_pos = next_word_pos
            check_and_insert()
            print("adj-noun worked :)")
        
    
    
    
    
def check_and_insert():
    # Criteria:

    if new_word_pos in allowed_pos:

        # find the index to insert the new word at.
        index_to_insert = len(result_tuples)
        for i, (word, index, pos) in enumerate(result_tuples): # this uses tuple unpacking and the enumerate function
            if new_word_index < index:
                index_to_insert = i
                break

                
                
        # check if the new_word_pos is different from the POS of the words before and after the insertion point. checks 2 things:
            # 1. does the word before the insertion point have a different POS, or is the new word gonna be the first word in the list.
            # 2. does the word after the insertion point have a different POS, or is the new word gonna be the last word in the list.
        if (index_to_insert == 0 or result_tuples[index_to_insert-1][2] != new_word_pos) and (index_to_insert == len(result_tuples) or result_tuples[index_to_insert][2] != new_word_pos):
            # insert the new word's tuple into the result_tuples list
            result_tuples.insert(index_to_insert, (new_word, new_word_index, new_word_pos))


    if len(result_tuples) == 7:
        
#         # force the last word to be a noun--not sure if this helps or not
#         last_word_pos = result_tuples[-1][2][:2]
#         if last_word_pos != "NN":
#             result_tuples.pop()
#             global pops
#             pops +=1

        # force the first word to not be a verb
        first_word_pos = result_tuples[0][2][:2]
        if first_word_pos == "VB":
            result_tuples.pop()
            global pops
            pops +=1
    
       
def main():
    
    text2tokens(text)
    while len(result_tuples) < 7 and pops < max_pops:
        get_word()
        check_and_insert()
        sequence_check()
    
main()   
    
    
# print(result_tuples)

result_words = []
result_index = [] # result_index and result_pos lists are for easy reference to ensure the criteria worked
result_pos = []
for result in result_tuples:
    result_words.append(result[0])
    result_index.append(result[1])
    result_pos.append(result[2])

print(f"\n{result_words}\n{result_index}\n{result_pos}\n\npops={pops}")

adj-noun worked :)
adj-noun worked :)
adj-noun worked :)
adj-noun worked :)
adj-noun worked :)
adj-noun worked :)
adj-noun worked :)
adj-noun worked :)
adj-noun worked :)

['Cultivated', 'clingstones', 'white', 'flesh', 'typically', 'yellow-fleshed']
[0, 5, 29, 30, 31, 39]
['VB', 'NN', 'JJ', 'NN', 'RB', 'JJ']

pops=25


In [32]:
from tkinter import *
import requests

text = "Cultivated peaches are divided into clingstones and freestones, depending on whether the flesh sticks to the stone or not; both can have either white or yellow flesh. Peaches with white flesh typically are very sweet with little acidity, while yellow-fleshed peaches typically have an acidic tang coupled with sweetness, though this also varies greatly. Both colors often have some red on their skins. Low-acid, white-fleshed peaches are the most popular kinds in China, Japan, and neighbouring Asian countries, while Europeans and North Americans have historically favoured the acidic, yellow-fleshed cultivars. Peach trees are relatively short-lived as compared with some other fruit trees. In some regions orchards are replanted after 8 to 10 years, while in others trees may produce satisfactorily for 20 to 25 years or more, depending upon their resistance to diseases, pests, and winter damage."

highlight = ['Cultivated', 'clingstones', 'white', 'flesh', 'typically', 'yellow-fleshed']
index = [0, 5, 29, 30, 31, 39]

# customtkinter.set_appearance_mode('light')
# customtkinter.set_default_color_theme("green")

poet = Tk()
poet.geometry("600x500")

# textframe = Frame(master = poet, width = 300, height = 400)
# textframe.place(relx=.5, rely=.5,anchor= 'center')

poemtext = Text(poet, wrap = "word", spacing1 = 1, spacing2 = 2, spacing3 = 1, width = 40)
poemtext.insert(INSERT,text)
scrolly = Scrollbar(poemtext)
poemtext.pack(anchor= 'center', expand = True)

# poemtext.tag_add("help","1.0", "1.4")
# poemtext.tag_config("help", background = "black")



for i in text:
    if i not in highlight:
        poemtext.tag_add(f"{i}","1.0", "1.26")
        poemtext.tag_config(f"{i}", background = "black")

#we have a problem with the tag add: it takes the indexes as floats: the whole number is the word index and the decimal number is the char index. We need a way to convert our existing indices into this type        
    
poet.mainloop()

In [9]:
pip install tkinter

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


V3:

In [1]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
import random
import re
import requests
import json
from tkinter import *


# for reference:
acronym_meanings = {
    'CC': 'Coordinating conjunction',
    'CD': 'Cardinal number',
    'DT': 'Determiner',
    'EX': 'Existential there',
    'FW': 'Foreign word',
    'IN': 'Preposition or subordinating conjunction',
    'JJ*': 'Adjective',
    'MD': 'Modal',
    'NN*': 'Noun',
    'PDT': 'Predeterminer',
    'POS': 'Possessive ending',
    'PRP*': 'Pronoun',
    'RB*': 'Adverb',
    'RP': 'Particle',
    'SYM': 'Symbol',
    'TO': 'To',
    'UH': 'Interjection',
    'VB*': 'Verb',
    'WDT': 'Wh-determiner',
    'WP*': 'Wh-pronoun',
    'WRB': 'Wh-adverb'
}


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NYCTA-P6\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\NYCTA-P6\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NYCTA-P6\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from tkinter import *
poet = Tk()
poet.geometry("600x500")


def search_wiki():
        global text
        user_input = input("Choose a topic to make blackout poetry out of! ").lower()

        try:
            try:
                text = wiki_pull(user_input)
                if text == "":
                    text = wiki_pull_backup(user_input)
                    print(f"Your search terms weren't a perfect match. Here's the closest article we found: {closest_match}\n")

            except:
                text = wiki_pull_backup(user_input)
                print(f"Your search terms weren't a perfect match. Here's the closest article we found: {closest_match}\n")
        except:
            print(f"\nYour search terms didn't match any results! Please try again")
            search_wiki()


def wiki_pull(user_input):
    wiki_api_url = "https://en.wikipedia.org/w/api.php"

    # parameters
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "exintro": "", # get intro section only
        "explaintext": "", # plain text
        "titles": user_input}

    # making request
    response = requests.get(wiki_api_url, params=params)
    response_json = response.json()

    # have to get page_id so we can index the json dictionary for the extract
    page_id = list(response_json["query"]["pages"].keys())[0]
    page_content = response_json["query"]["pages"][page_id]["extract"]

    return page_content



def wiki_pull_backup(user_topic):
    
    wiki_api_url = "https://en.wikipedia.org/w/api.php"

    # using search action instead of query
    params = {
        "action": "query",
        "format": "json",
        "list": "search",
        "srsearch": user_topic,
        "srprop": "",
        "utf8": ""}

    # making request
    response = requests.get(wiki_api_url, params=params)
    response_json = response.json()

    # selecting the page with closest match
    global closest_match
    closest_match = response_json["query"]["search"][0]["title"]

    # using the selected title for content request
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "exintro": "", # get intro section only
        "explaintext": "", # plain text
        "titles": closest_match}

    # making request for content
    response = requests.get(wiki_api_url, params=params)
    response_json = response.json()

    # have to get page_id so we can index the json dictionary for the extract
    page_id = list(response_json["query"]["pages"].keys())[0]
    page_content = response_json["query"]["pages"][page_id]["extract"]

    return page_content







# the parts of speech that are allowed in the output
allowed_pos = ["JJ", "RB", "VB", "NN"]


result_tuples = []
chosen = []

# pops is used to limit the number of loops. Sometimes it bugs out if max_pops is set too high.
max_pops = 25
global pops
pops = 0




def text2tokens(text):
    global tokens, text_copy, text_offset
    tokens = word_tokenize(text)
    
    # to preserve a version of the original text
    text_copy = text
    
    # fixing \n offset
    text_offset = text.replace("\n", " \n")
    
    # attaching the index from the tokenized text AND the original text to each word (both are later added to result_tuple)
    # note: i think \n's in the text source offset the text_index by 1 character
    global both_indices
    both_indices = []
    for i, token in enumerate(tokens):
        text_index = text_copy.find(token) # searches for substring in string
        text_index_end = text_index + (len(token)-1)
        text_copy = text_copy.replace(token, "x" * len(token), 1) # replace the current token with all x's
        both_indices.append((token, i, text_index, text_index_end))
    
    

# not sure if stopwords are good or bad--a lot are already avoided due to "allowed_pos"
#     stop_words = set(stopwords.words('english'))
#     bad_tokens = []
#     for word in bad_tokens:
#         if word.lower() not in stop_words:
#             bad_tokens.append(word)



def get_word():
    global choice, new_word, new_word_index, new_word_pos
    choice = random.choice(range(len(tokens)))
    chosen.append(choice)
    new_word = tokens[choice]
    new_word_index = choice
    new_word_pos = pos_tag([new_word])[0][1][:2] # syntax: pos_tag generates a list of a tuple. This grabs the tuple, then grabs the 2nd element which is the POS as a string, then grabs the first 2 letters.
    
    
    
# this function is generalizable to any POS sequences i believe
def sequence_check():
    """Checks to see if there is an sequence like 'adjective then noun' around the chosen word. 
    If so, then it adds both words to the results list."""
    
    global new_word, new_word_index, new_word_pos
    if choice > 0 and choice < len(tokens) - 1: # avoids list index out of range errors
        prior_word = tokens[choice-1]
        prior_word_index = choice-1
        prior_word_pos = pos_tag([prior_word])[0][1][:2]

        next_word = tokens[choice+1]
        next_word_index = choice+1
        next_word_pos = pos_tag([next_word])[0][1][:2]
    
    # adj-noun-verb
        if prior_word_pos == "JJ" and new_word_pos == "NN" and next_word_pos =="VB":
            new_word = prior_word
            new_word_index = prior_word_index
            new_word_pos = prior_word_pos
            check_and_insert()
            new_word = next_word
            new_word_index = next_word_index
            new_word_pos = next_word_pos
            check_and_insert()
            print("adj-noun-verb worked :)")
    
    # adj-noun
        elif prior_word_pos == "JJ" and new_word_pos == "NN":
            new_word = prior_word
            new_word_index = prior_word_index
            new_word_pos = prior_word_pos
            check_and_insert()
            print("adj-noun worked :)")

        elif new_word_pos == "JJ" and next_word_pos == "NN":
            new_word = next_word
            new_word_index = next_word_index
            new_word_pos = next_word_pos
            check_and_insert()
            print("adj-noun worked :)")
        
    
    
    
    
def check_and_insert():
    # Criteria:

    # regular expression that matches only word characters. Prevents stuff like "]" which is a noun for some reason.
    pattern = re.compile(r'\w+')
    if pattern.match(new_word):
    
    
    
        if new_word_pos in allowed_pos:

            # find the index to insert the new word at.
            index_to_insert = len(result_tuples)
            for i, (word, index, pos, text_index, text_index_end) in enumerate(result_tuples): # this uses tuple unpacking and the enumerate function
                if new_word_index < index:
                    index_to_insert = i
                    break



            # check if the new_word_pos is different from the POS of the words before and after the insertion point. checks 2 things:
                # 1. does the word before the insertion point have a different POS, or is the new word gonna be the first word in the list.
                # 2. does the word after the insertion point have a different POS, or is the new word gonna be the last word in the list.
            if (index_to_insert == 0 or result_tuples[index_to_insert-1][2] != new_word_pos) and (index_to_insert == len(result_tuples) or result_tuples[index_to_insert][2] != new_word_pos):
                
                # getting index from original text
                for (token, token_index, text_index, text_index_end) in both_indices:
                    if new_word_index == token_index:
                        new_text_index = text_index
                        new_text_index_end = text_index_end
                
                # insert the new word's tuple into the result_tuples list
                result_tuples.insert(index_to_insert, (new_word, new_word_index, new_word_pos, new_text_index, new_text_index_end))


        if len(result_tuples) == 7:

    #         # force the last word to be a noun--not sure if this helps or not
    #         last_word_pos = result_tuples[-1][2][:2]
    #         if last_word_pos != "NN":
    #             result_tuples.pop()
    #             global pops
    #             pops +=1

            # force the first word to not be a verb
            first_word_pos = result_tuples[0][2][:2]
            if first_word_pos == "VB":
                result_tuples.pop()
                global pops
                pops +=1
    

poemtext = Text(poet, wrap = "word", spacing1 = 1, spacing2 = 2, spacing3 = 1, width = 40)    
    
    
while_counter = 0    
def main():
    search_wiki()
    
    text2tokens(text)
    
    poemtext.insert(INSERT,text)
    poemtext.config(state = DISABLED)
    scrolly = Scrollbar(poemtext)
    poemtext.pack(anchor= 'center', expand = True)
    
    while_counter = 0
    while len(result_tuples) < 7 and pops < max_pops and while_counter < 150:
        get_word()
        check_and_insert()
        sequence_check()
        
        while_counter +=1
        
    
    
#
    

result_words = []
result_index = [] # result_index and result_pos lists are for easy reference to ensure the criteria worked
result_pos = []
result_text_index = []
for (word, index, pos, text_index, text_index_end) in result_tuples:
    result_words.append(word)
    result_index.append(index)
    result_pos.append(pos)
    result_text_index.append((text_index, text_index_end))


    
    
    
#Button for Refresh
refresh = Button(poet,text = "Refresh", command = main)
refresh.pack(anchor = "w")

#Button to cause the blackout?


#poemtext = Text(poet, wrap = "word", spacing1 = 1, spacing2 = 2, spacing3 = 1, width = 40)
# poemtext.insert(INSERT,text)
# poemtext.config(state = DISABLED)
# scrolly = Scrollbar(poemtext)
# poemtext.pack(anchor= 'center', expand = True)






#print(f"\n{result_words}\n{result_index}\n{result_pos}\n{result_text_index}\n\npops={pops}\n\n{text_offset}\n")

poet.mainloop()

Choose a topic to make blackout poetry out of! jam

Your search terms didn't match any results! Please try again
